# Programa para leer y procesar los miserables

## Leamos el libro en formato epub y guardemoslo en CSV

In [ ]:
## Instalar dependencias
!pip install ebooklib pandas bs4 unidecode nltk

In [ ]:
from ebooklib import epub
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
#  Subir el archivo EPUB a Google Colab

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Asegurar que ITEM_DOCUMENT está definido
DOCUMENT_TYPE = epub.ITEM_DOCUMENT if hasattr(epub, "ITEM_DOCUMENT") else 9

def extract_text_from_epub(epub_path):
    book = epub.read_epub(epub_path)
    text_data = []

    for item in book.get_items():
        if item.get_type() == DOCUMENT_TYPE:
            soup = BeautifulSoup(item.get_content(), "html.parser")
            paragraphs = soup.find_all("p")
            for paragraph in paragraphs:
                text_data.append(paragraph.get_text(strip=True))

    return text_data

def save_text_to_csv(text_data, csv_path):
    df = pd.DataFrame(text_data, columns=["Text"])
    df.to_csv(csv_path, index=False, encoding="utf-8")



In [ ]:
import glob
#No encontraba la ruta de donde estaba guardado el e-book y como mandarlo llamar
# Buscar archivos EPUB en toda tu unidad de Google Drive
epub_files = glob.glob("/content/drive/My Drive/**/*.epub", recursive=True)

# Mostrar archivos encontrados
print(epub_files)

['/content/drive/My Drive/Colab Notebooks/datos/Los-miserables.epub']


In [ ]:
# Ruta del archivo en Google Drive
epub_path = "/content/drive/My Drive/Colab Notebooks/datos/Los-miserables.epub"
csv_path = "/content/drive/My Drive/Colab Notebooks/datos/Los-miserables.csv"

# Convertir EPUB a CSV
text_data = extract_text_from_epub(epub_path)
save_text_to_csv(text_data, csv_path)

print(f"Archivo guardado en: {csv_path}")

/usr/local/lib/python3.11/dist-packages/ebooklib/epub.py:1395: UserWarning: In the future version we will turn default option ignore_ncx to True.
  warnings.warn('In the future version we will turn default option ignore_ncx to True.')
/usr/local/lib/python3.11/dist-packages/ebooklib/epub.py:1423: FutureWarning: This search incorrectly ignores the root element, and will be fixed in a future version.  If you rely on the current behaviour, change it to './/xmlns:rootfile[@media-type]'
  for root_file in tree.findall('//xmlns:rootfile[@media-type]', namespaces={'xmlns': NAMESPACES['CONTAINERNS']}):


Archivo guardado en: /content/drive/My Drive/Colab Notebooks/datos/Los-miserables.csv


## Leer el csv y limpiarlo


In [ ]:
import nltk
### La descarga de las stopwords solo hay que hacerla una vez
nltk.download("stopwords")
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
import pandas as pd
import re
import unidecode
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
# Cargar stopwords en español
stop_words = set(stopwords.words("spanish"))


In [ ]:
### Funcion para limpiar el texto
def clean_text(text):
    if not isinstance(text, str):  # Si el valor no es string, convertirlo
        return ""

    text = text.lower()  # Convertir a minúsculas
    text = unidecode.unidecode(text)  # Eliminar acentos
    text = re.sub(r"\s+", " ", text)  # Remover espacios extra
    text = text.translate(str.maketrans("", "", string.punctuation))  # Eliminar puntuación
    text = text.strip()  # Eliminar espacios al inicio y al final

    # Eliminar stopwords en español
    #words = text.split()
    #words = [word for word in words if word not in stop_words]
    #text = " ".join(words)

    return text


In [ ]:
# Cargar el CSV que generamos con "Los Miserables"

csv_path = "/content/drive/My Drive/Colab Notebooks/datos/Los-miserables.csv"
df = pd.read_csv(csv_path)
### Los primeros 5 renglones
df.head(5)

,Text
0,NaN
1,Los miserables
2,"Hugo, Victor"
3,Novela
4,NaN


In [ ]:
## Los últimos 5 renglones
df.tail()

,Text
3022,Una última palabra sobre Fantine.
3023,"Todos nosotros tenemos una madre, la tierra. F..."
3024,"El cura creyó obrar bien, y posiblemente obró ..."
3025,"Fantine fue, pues, enterrada en el rincón grat..."
3026,[FIN DE LA PRIMERA PARTE]


In [ ]:
# Aplicar limpieza a la columna "Text"
df["Text"] = df["Text"].astype(str).apply(clean_text)

In [ ]:
df.head(5)

,Text
0,nan
1,los miserables
2,hugo victor
3,novela
4,nan



 Ahora hay que tokenizar

In [ ]:
def tokenize(text):
    # Tokenizar en palabras
    tokens = word_tokenize(text)

    # Filtrar stopwords
    #tokens = [word for word in tokens if word not in stop_words]

    return tokens

df["Tokens"] = df["Text"].astype(str).apply(tokenize)


def tokenize_and_remove_stop_words(text):
    # Tokenizar en palabras
    tokens = word_tokenize(text)

    # Filtrar stopwords
    tokens = [word for word in tokens if word not in stop_words]

    return tokens

df["Tokens"] = df["Text"].astype(str).apply(tokenize)
df["Tokens_no_stopwords"] = df["Text"].astype(str).apply(tokenize_and_remove_stop_words)

In [ ]:
df.head(5)


,Text,Tokens,Tokens_no_stopwords
0,nan,[nan],[nan]
1,los miserables,"[los, miserables]",[miserables]
2,hugo victor,"[hugo, victor]","[hugo, victor]"
3,novela,[novela],[novela]
4,nan,[nan],[nan]


In [ ]:
df.tail(5)

,Text,Tokens,Tokens_no_stopwords
3022,una ultima palabra sobre fantine,"[una, ultima, palabra, sobre, fantine]","[ultima, palabra, fantine]"
3023,todos nosotros tenemos una madre la tierra fan...,"[todos, nosotros, tenemos, una, madre, la, tie...","[madre, tierra, fantine, devuelta, madre]"
3024,el cura creyo obrar bien y posiblemente obro b...,"[el, cura, creyo, obrar, bien, y, posiblemente...","[cura, creyo, obrar, bien, posiblemente, obro,..."
3025,fantine fue pues enterrada en el rincon gratui...,"[fantine, fue, pues, enterrada, en, el, rincon...","[fantine, pues, enterrada, rincon, gratuito, c..."
3026,fin de la primera parte,"[fin, de, la, primera, parte]","[fin, primera, parte]"


In [ ]:
# Construir el vocabulario para el total
vocabulary = {}  # Diccionario de vocabulario
for tokens in df["Tokens"]:
    for token in tokens:
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)  # Asignar un índice único


In [ ]:
vocabulary

{'nan': 0,
 'los': 1,
 'miserables': 2,
 'hugo': 3,
 'victor': 4,
 'novela': 5,
 'se': 6,
 'reconocen': 7,
 'derechos': 8,
 'morales': 9,
 'de': 10,
 'obra': 11,
 'dominio': 12,
 'publico': 13,
 'distribucion': 14,
 'gratuita': 15,
 'prohibida': 16,
 'su': 17,
 'venta': 18,
 'y': 19,
 'en': 20,
 'medios': 21,
 'ajenos': 22,
 'a': 23,
 'la': 24,
 'fundacion': 25,
 'carlos': 26,
 'slim': 27,
 'lago': 28,
 'zurich': 29,
 'plaza': 30,
 'carso': 31,
 'ii': 32,
 'piso': 33,
 '5': 34,
 'col': 35,
 'ampliacion': 36,
 'granada': 37,
 'c': 38,
 'p': 39,
 '11529': 40,
 'ciudad': 41,
 'mexico': 42,
 'contactopruebatorg': 43,
 'fantine': 44,
 'un': 45,
 'justo': 46,
 'el': 47,
 'senor': 48,
 'myriel': 49,
 '1815': 50,
 'monsenor': 51,
 'charlesfrancoisbienvenu': 52,
 'era': 53,
 'obispo': 54,
 'digne': 55,
 'anciano': 56,
 'cerca': 57,
 'setenta': 58,
 'cinco': 59,
 'anos': 60,
 'ocupaba': 61,
 'sede': 62,
 'desde': 63,
 '1806': 64,
 'aunque': 65,
 'este': 66,
 'detalle': 67,
 'no': 68,
 'interesa'

In [ ]:
# Guardar vocabulario en Parquet
vocab_df = pd.DataFrame(list(vocabulary.items()), columns=["Word", "Index"])
vocab_parquet_path = "/content/drive/My Drive/Colab Notebooks/datos/vocabulary.parquet"
vocab_df.to_parquet(vocab_parquet_path, index=False)

In [ ]:
import collections #Para usar el contador de palabras

In [ ]:
# Contador de palabras basado en el vocabulario
word_counts = collections.Counter()
word_counts_nostop_words = collections.Counter()

# Contar frecuencia solo para palabras en el vocabulario
for tokens in df["Tokens"]:
    for token in tokens:
        if token in vocabulary:
            word_counts[token] += 1

# Contar frecuencia solo para palabras en el vocabulario
for tokens in df["Tokens_no_stopwords"]:
    for token in tokens:
        if token in vocabulary:
            word_counts_nostop_words[token] += 1

#Calcular estadísticas considerando todas las palabras
total_words = sum(word_counts.values())  # Total de palabras en el texto
unique_words = len(word_counts)  # Palabras únicas en el vocabulario
most_common_words = word_counts.most_common(100)  # 100 más frecuentes
least_common_words = word_counts.most_common()[-100:]  # 100 menos frecuentes

#Calcular estadísticas quitando stop_words
total_words_no_sw = sum(word_counts_nostop_words.values())  # Total de palabras en el texto
unique_words_no_sw = len(word_counts_nostop_words)  # Palabras únicas en el vocabulario
most_common_words_no_sw = word_counts_nostop_words.most_common(100)  # 100 más frecuentes
least_common_words_no_sw = word_counts_nostop_words.most_common()[-100:]  # 100 menos frecuentes

Total con stopwords

In [ ]:
# Imprimir resultados
print(f" Total de palabras en el libro: {total_words}")
print(f" Número de palabras únicas en el vocabulario: {unique_words}")
print("\n 100 palabras más frecuentes:")
print(most_common_words)
print("\n100 palabras menos frecuentes:")
print(least_common_words)

 Total de palabras en el libro: 109287
 Número de palabras únicas en el vocabulario: 13165

 100 palabras más frecuentes:
[('de', 5325), ('la', 3918), ('que', 3818), ('el', 3394), ('y', 3121), ('en', 2836), ('a', 2488), ('se', 1681), ('un', 1601), ('no', 1498), ('los', 1353), ('una', 1319), ('su', 1245), ('las', 935), ('por', 935), ('con', 924), ('habia', 857), ('del', 813), ('al', 755), ('es', 749), ('lo', 719), ('le', 667), ('era', 650), ('como', 572), ('mas', 513), ('para', 503), ('senor', 447), ('esta', 414), ('pero', 372), ('hombre', 363), ('si', 358), ('sus', 344), ('todo', 327), ('me', 326), ('sin', 311), ('obispo', 286), ('dijo', 281), ('cuando', 274), ('estaba', 273), ('sobre', 269), ('dos', 264), ('este', 261), ('aquel', 253), ('mi', 244), ('ya', 229), ('hacia', 219), ('yo', 218), ('esto', 218), ('madeleine', 214), ('tenia', 212), ('jean', 200), ('ha', 199), ('fantine', 194), ('valjean', 192), ('aquella', 190), ('hay', 186), ('he', 182), ('ser', 181), ('muy', 178), ('javert',

Total sin stopwords

In [ ]:
# Imprimir resultados
print(f" Total de palabras en el libro sin considerar stopwords: {total_words_no_sw}")
print(f" Número de palabras únicas en el vocabulario sin considerar stopwords: {unique_words_no_sw}")
print("\n 100 palabras más frecuentes sin considerar stopwords:")
print(most_common_words_no_sw)
print("\n100 palabras menos frecuentes sin considerar stopwords:")
print(least_common_words_no_sw)

 Total de palabras en el libro: 57219
 Número de palabras únicas en el vocabulario: 12991

 100 palabras más frecuentes:
[('habia', 857), ('mas', 513), ('senor', 447), ('hombre', 363), ('si', 358), ('obispo', 286), ('dijo', 281), ('dos', 264), ('aquel', 253), ('hacia', 219), ('madeleine', 214), ('tenia', 212), ('jean', 200), ('fantine', 194), ('valjean', 192), ('aquella', 190), ('ser', 181), ('javert', 175), ('mismo', 173), ('tan', 158), ('bien', 157), ('alcalde', 149), ('vez', 148), ('despues', 146), ('puerta', 137), ('anos', 136), ('dios', 130), ('mujer', 127), ('momento', 125), ('tiempo', 124), ('sido', 124), ('casa', 123), ('aqui', 120), ('noche', 119), ('hecho', 118), ('tres', 115), ('dia', 114), ('luego', 113), ('cabeza', 113), ('decir', 112), ('voz', 111), ('alli', 107), ('ojos', 107), ('monsenor', 105), ('aun', 105), ('todas', 104), ('vida', 102), ('parecia', 101), ('senora', 100), ('solo', 98), ('bajo', 98), ('volvio', 93), ('pues', 93), ('gran', 92), ('hacer', 92), ('entonces